In [293]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [294]:
import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

# Asumiendo que es una ocupacion del 80% de camas
INDICE_OCUPACION_CAMAS = 1.25

# Agrega las columnas de poblacion de interes
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

# Proyección de casos INT al 2035

En este cuadernillo se quiere estimar la deminada que atenderá el INT para el 2035. Con esta demanada se calcularán las cantidades de camas necesarias para suplir tal demanda. Para determinar la demanda se utilizarán:

1. Cantidad de casos teóricos por problemas de salud relevantes seleccionados: Estos se calcularon según la incidencia de cada problema de salud y la proyección de la población al 2035.

2. Cantidad de casos atendidos actualmente por cada problema de salud: Estos se obtuvieron al analizar la base de egresos hospitalarios DEIS.

Con ambos datos, se obtendrá cuanto % de atención ha cubierto el INT de los casos teóricos.

In [295]:
# Obtiene los casos de area de influencia
casos_area_de_influencia = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="casos_area_de_influencia_INT"
)

# Preprocesa el diagnostico
casos_area_de_influencia["Diagnostico"] = (
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0]
)

# Preprocesa los diagnosticos agrupados
casos_area_de_influencia["Diagnosticos Contenidos"] = casos_area_de_influencia[
    "Diagnosticos Contenidos"
].str.split(", ")

# Renombra columnas de la poblacion
casos_area_de_influencia = casos_area_de_influencia.rename(columns=COLUMNAS_POBLACION_INTERES)

In [296]:
# Obtiene los diagnosticos relevantes a filtrar de los egresos hospitalarios
DIAGNOSTICOS_RELEVANTES = list(
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0].unique()
)

# Obtiene los diagnosticos agrupados
DIAGNOSTICOS_AGRUPADOS = list(
    casos_area_de_influencia.explode("Diagnosticos Contenidos")["Diagnosticos Contenidos"]
    .dropna()
    .str.strip()
    .unique()
)

# Suma los diagnosticos agrupados a los diagnosticos relevantes
DIAGNOSTICOS_RELEVANTES += DIAGNOSTICOS_AGRUPADOS
DIAGNOSTICOS_RELEVANTES = sorted(DIAGNOSTICOS_RELEVANTES)

In [297]:
# Genera duplas de diagnosticos a reasignar
DIAGNOSTICOS_A_REASIGNAR = casos_area_de_influencia.query("`Diagnosticos Contenidos`.notna()")[
    ["Diagnostico", "Diagnosticos Contenidos"]
]

In [298]:
casos_area_de_influencia = casos_area_de_influencia.set_index("Diagnostico")

## Obtención de casos atendidos entre 2017 y 2020 para diagnósticos relevantes.

Estos se obtendrán del análisis de egresos hospitalarios a nivel país DEIS.

In [299]:
# Lee los egresos del Torax
egresos_torax = pl.read_csv("../data/raw/6_egresos_deis/egresos_procesados_112103.csv")

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [300]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_torax = egresos_torax.filter(
    (pl.col("DIAG1").is_in(DIAGNOSTICOS_RELEVANTES))
)

## Reasginación de diagnósticos y agrupación

En este apartado se reasignarán diagnósticos de la base de datos DEIS a un grupo mayor. Por ejemplo,
los siguientes diagnósticos:

- C341 - Tumor maligno del lóbulo superior, bronquio o pulmón
- C342 - Tumor maligno del lóbulo medio, bronquio o pulmón
- C343 - Tumor maligno del lóbulo inferior, bronquio o pulmón
- C780 - Tumor maligno secundario del pulmón
- C782 - Tumor maligno secundario de la pleura
- D381 - Tumor de comportamiento incierto o desconocido de la tráquea, de los bronquios y del pulmón

Todos estos se reasignarán al diagnóstico C34XN (N, para que no se mezclen con los C34X asignados realmente así). Esto, con el fin de poder calcular las estadísticas agrupadas para todos estos diagnósticos (egresos por persona, días de estada).

In [301]:
for row in DIAGNOSTICOS_A_REASIGNAR.itertuples():
    diagnostico_nuevo = row[1]
    diagnosticos_antiguos = row[2]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    egresos_mas_relevantes_torax = egresos_mas_relevantes_torax.with_columns(
        pl.col("DIAG1").replace(diagnosticos_antiguos, diagnostico_nuevo)
    )

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['I052', 'I080', 'I081'] a I051
Cambiando ['I352'] a I350
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q676', 'Q677', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX


In [302]:
# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_torax = (
    build_features.calcular_resumen_metricas_desagregadas_y_agrupadas_en_anios(
        egresos_mas_relevantes_torax, 2017, 2019
    )
)

## Estimación de casos Hospitalizados utilizando % de Hospitalización del INT

En este caso, se utilizará el % de hospitalización del INT de los pacientes ambulatorios.

In [303]:
PORCENTAJE_HOSPITALIZADOS = pd.read_excel(
    "../data/external/incidencias_y_prevalencias_INT.xlsx",
    usecols=["Diagnostico", "Porcentaje Hospitalizados de Ambulatorios"],
)

PORCENTAJE_HOSPITALIZADOS = PORCENTAJE_HOSPITALIZADOS.drop_duplicates("Diagnostico")

PORCENTAJE_HOSPITALIZADOS["Diagnostico"] = (
    PORCENTAJE_HOSPITALIZADOS["Diagnostico"].str.split(" - ").str[0]
)
PORCENTAJE_HOSPITALIZADOS = PORCENTAJE_HOSPITALIZADOS.set_index("Diagnostico")[
    "Porcentaje Hospitalizados de Ambulatorios"
]

In [304]:
casos_INT_hospitalizados_porcentaje_amb = casos_area_de_influencia[
    COLUMNAS_POBLACION_INTERES.values()
].mul(PORCENTAJE_HOSPITALIZADOS, axis=0)

# Se obtiene la cantidad de egresos que debiese tener el INT, asumiendo que trabajara con la misma
# eficiencia
egresos_estimados_INT_porcentaje_amb = casos_INT_hospitalizados_porcentaje_amb.mul(
    metricas_diags_relevantes_torax[("agrupado_entre_2017_2019", "egresos_por_paciente")], axis=0
)

# Se obtiene la cantidad de dias de estada que debiese el INT, asumiendo que trabajara con la misma
# eficiencia
dias_estada_estimados_int_porcentaje_amb = egresos_estimados_INT_porcentaje_amb.mul(
    metricas_diags_relevantes_torax[("agrupado_entre_2017_2019", "dias_estada_promedio")], axis=0
)

# Estima la cantidad de camas necesaarias por diagnostico
camas_estimadas_int_porcentaje_amb = (
    dias_estada_estimados_int_porcentaje_amb / 365.25
) * INDICE_OCUPACION_CAMAS

# Calcula las camas totales necesarias
camas_totales_int_porcentaje_amb = camas_estimadas_int_porcentaje_amb.sum()

In [305]:
print(camas_totales_int_porcentaje_amb)

2017    172.646420
2018    175.811053
2019    179.535920
2020    183.335573
2021    185.570719
2022    186.947920
2023    188.093427
2024    189.155604
2025    190.164358
2026    191.125629
2027    192.040079
2028    192.908007
2029    193.727719
2030    194.497414
2031    195.299338
2032    196.015811
2033    196.687303
2034    197.316900
2035    197.904998
dtype: float64


In [306]:
# Se quiere tener un 60% de UPC (1:2, UCI:UTI, y 40% de Medias)
FRACCION_UCI = 0.2
FRACCION_UTI = 0.4
FRACCION_MEDIAS = 0.4

# Se definen los multiplos de las camas segun norma
MULTIPLOS_DE_CAMAS_UCI = 1
MULTIPLOS_DE_CAMAS_UTI = 2
MULTIPLOS_DE_CAMAS_MEDIAS = 24

# Agrega las camas necesarias para trasplantes a las del 2035
camas_al_2035 = camas_totales_int_porcentaje_amb[2035] + 7.91569255

In [307]:
# Obtiene la cantidad de camas segun los porcentaje UCI, UTI y Medias
camas_por_porcentaje = build_features.obtener_camas(
    camas_al_2035, FRACCION_UCI, FRACCION_UTI, FRACCION_MEDIAS
)

# Ajusta las camas segun norma y hacia arriba
camas_ajustada_con_redondeo_comun = (
    build_features.calcular_camas_ajustadas(
        camas_al_2035,
        FRACCION_UCI,
        FRACCION_UTI,
        FRACCION_MEDIAS,
        MULTIPLOS_DE_CAMAS_UCI,
        MULTIPLOS_DE_CAMAS_UTI,
        MULTIPLOS_DE_CAMAS_MEDIAS,
        "round"
    )
)

# Ajusta las camas segun norma y hacia arriba
camas_ajustada_con_redondeo_hacia_arriba = (
    build_features.calcular_camas_ajustadas(
        camas_al_2035,
        FRACCION_UCI,
        FRACCION_UTI,
        FRACCION_MEDIAS,
        MULTIPLOS_DE_CAMAS_UCI,
        MULTIPLOS_DE_CAMAS_UTI,
        MULTIPLOS_DE_CAMAS_MEDIAS,
        "ceil"
    )
)

In [308]:
resumen_final_camas = pd.DataFrame(
    {
        "tipo_de_cama": ["Totales", "UCI", "UTI", "Medias"],
        "camas_totales_al_2035": camas_por_porcentaje,
        "camas_ajustadas_a_norma_redondeo_comun_al_2035": camas_ajustada_con_redondeo_comun,
        "camas_ajustada_con_redondeo_hacia_arriba_al_2035": camas_ajustada_con_redondeo_hacia_arriba,
    }
).set_index("tipo_de_cama")

In [309]:
display(resumen_final_camas)

,camas_totales_al_2035,camas_ajustadas_a_norma_redondeo_comun_al_2035,camas_ajustada_con_redondeo_hacia_arriba_al_2035
tipo_de_cama,,,
Totales,205.820690,195.0,222.0
UCI,41.164138,41.0,42.0
UTI,82.328276,82.0,84.0
Medias,82.328276,72.0,96.0


## Obtener resumen para MINSAL

In [310]:
resumen_metricas_hosp = metricas_diags_relevantes_torax[
    [
        ("agrupado_entre_2017_2019", "egresos_por_paciente"),
        ("agrupado_entre_2017_2019", "dias_estada_promedio"),
    ]
]

resumen_metricas_hosp.columns = [
    "egresos_por_paciente_agrupado_2017_a_2019",
    "dias_estada_promedio_agrupado_2017_a_2019",
]
resumen_casos = casos_INT_hospitalizados_porcentaje_amb[2035]
resumen_casos.name = "casos_hospitalizados_2035"

resumen_egresos = egresos_estimados_INT_porcentaje_amb[2035]
resumen_egresos.name = "egresos_2035"

resumen_dias_estada = dias_estada_estimados_int_porcentaje_amb[2035]
resumen_dias_estada.name = "dias_estada_totales_2035"

resumen_camas = camas_estimadas_int_porcentaje_amb[2035]
resumen_camas.name = "camas_totales_2035"

resumen_total_hosp = (
    resumen_metricas_hosp.merge(
        resumen_casos,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_egresos,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_dias_estada,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_camas,
        how="left",
        left_index=True,
        right_index=True,
    )
)

ORDEN_COLUMNAS_RESUMEN = [
    "casos_hospitalizados_2035",
    "egresos_por_paciente_agrupado_2017_a_2019",
    "egresos_2035",
    "dias_estada_promedio_agrupado_2017_a_2019",
    "dias_estada_totales_2035",
    "camas_totales_2035",
]

resumen_total_hosp = resumen_total_hosp[ORDEN_COLUMNAS_RESUMEN]

Finalmente, une el resumen de casos teoricos desde el cuadernillo 1.0 con el resumen de este cuadernillo. Esto obtiene la tabla final para MINSAL.

In [311]:
resumen_casos_teoricos = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_INT"
)

resumen_casos_teoricos["Diagnostico_codigo"] = (
    resumen_casos_teoricos["Diagnostico"].str.split(" - ").str[0]
)
resumen_casos_teoricos = resumen_casos_teoricos.set_index("Diagnostico_codigo")

resumen_minsal = resumen_casos_teoricos.merge(
    resumen_total_hosp, how="right", left_index=True, right_index=True
)

resumen_minsal = resumen_minsal.reset_index(drop=True).set_index("Diagnostico")

## Guardar archivos

In [312]:
archivos_a_guardar = {
    "metricas_relevantes_INT": metricas_diags_relevantes_torax,
    "casos_hospitalizados_INT": casos_INT_hospitalizados_porcentaje_amb,
    "egresos_estimados_INT": egresos_estimados_INT_porcentaje_amb,
    "dias_estada_estimados_INT": dias_estada_estimados_int_porcentaje_amb,
    "camas_estimadas_desglosadas_INT": camas_estimadas_int_porcentaje_amb,
    "camas_totales_INT": camas_totales_int_porcentaje_amb,
    "resumen_total_hosp_INT": resumen_total_hosp,
    "resumen_MINSAL": resumen_minsal,
    "camas_totales_INT_ajustadas": resumen_final_camas,
}

with pd.ExcelWriter("../data/interim/estimacion_atencion_cerrada_INT.xlsx") as file:
    for nombre_hoja, df_a_guardar in archivos_a_guardar.items():
        print(f"Guardando {nombre_hoja}")
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)

Guardando metricas_relevantes_INT
Guardando casos_hospitalizados_INT
Guardando egresos_estimados_INT
Guardando dias_estada_estimados_INT
Guardando camas_estimadas_desglosadas_INT
Guardando camas_totales_INT
Guardando resumen_total_hosp_INT
Guardando resumen_MINSAL
Guardando camas_totales_INT_ajustadas
